In [1]:
import pandas as pd
import numpy as np
%matplotlib inline 
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('ggplot')
from pandas_profiling import ProfileReport

In [64]:
recipes=pd.read_csv(r'C:\Users\ejfel\OneDrive\Desktop\Metis\Projects\stout_vs_porter.csv')
recipes=recipes.fillna(0)

In [78]:
recipes=recipes[:16]
recipes

,recipe,style,IBU,SRM,OG,FG,Mash,flaked,light_adjunct,dark_adjunct,...,black,brown,chocolate,crystal_150,roasted_barley,special,candi_180,carafa2,aromatic,total
0,byo_porter,0.0,35.0,50.0,58.0,14.0,153.0,0.000000,0.324324,0.027027,...,0.027027,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.00000,9.250000
1,gf_porter,0.0,35.0,0.0,63.0,16.0,153.0,0.000000,0.074074,0.259259,...,0.000000,0.222222,0.037037,0.000000,0.000000,0.0,0.0,0.00000,0.00000,13.500000
2,bab_porter,0.0,40.0,0.0,0.0,0.0,153.0,0.000000,0.206897,0.103448,...,0.034483,0.000000,0.068966,0.000000,0.000000,0.0,0.0,0.00000,0.00000,14.500000
3,bs_porter,0.0,32.6,27.8,45.0,15.0,0.0,0.000000,0.114286,0.085714,...,0.028571,0.000000,0.057143,0.000000,0.000000,0.0,0.0,0.00000,0.00000,8.750000
4,la_porter,0.0,269.0,35.7,65.0,0.0,152.0,0.000000,0.040783,0.143556,...,0.000000,0.000000,0.051387,0.081566,0.010604,0.0,0.0,0.00000,0.00000,12.260000
5,pliers_porter,0.0,30.0,0.0,53.0,12.0,152.0,0.000000,0.090909,0.181818,...,0.000000,0.090909,0.090909,0.000000,0.000000,0.0,0.0,0.00000,0.00000,11.000000
6,aha_porter,0.0,37.0,35.6,69.0,12.0,156.0,0.000000,0.050000,0.083333,...,0.000000,0.000000,0.083333,0.000000,0.000000,0.0,0.0,0.00000,0.00000,30.000000
7,jamil_porter,0.0,34.1,35.7,65.0,0.0,152.0,0.000000,0.190476,0.079365,...,0.031746,0.000000,0.047619,0.000000,0.000000,0.0,0.0,0.00000,0.00000,15.750000
8,aab_stout,1.0,35.0,0.0,60.0,0.0,0.0,0.200000,0.050000,0.050000,...,0.000000,0.000000,0.050000,0.000000,0.000000,0.0,0.0,0.00000,0.00000,10.000000
9,byo_stout,1.0,73.0,48.0,72.0,17.0,154.0,0.000000,0.043409,0.100639,...,0.057429,0.000000,0.043211,0.000000,0.000000,0.0,0.0,0.00000,0.00000,15.736875


In [79]:
recipes.Base=(recipes.Base/recipes.total)
recipes.flaked=(recipes.flaked/recipes.total)
recipes.light_adjunct=(recipes.light_adjunct/recipes.total)
recipes.dark_adjunct=(recipes.dark_adjunct/recipes.total)
recipes.munich_8=(recipes.munich_8/recipes.total)
recipes.crystal_50=(recipes.crystal_50/recipes.total)
recipes.crystal_80=(recipes.crystal_80/recipes.total)
recipes.crystal_150=(recipes.crystal_150/recipes.total)
recipes.black=(recipes.black/recipes.total)
recipes.brown=(recipes.brown/recipes.total)
recipes.chocolate=(recipes.chocolate/recipes.total)
recipes.roasted_barley=(recipes.roasted_barley/recipes.total)
recipes.special=(recipes.special/recipes.total)
recipes.candi_180=(recipes.candi_180/recipes.total)
recipes.carafa2=(recipes.carafa2/recipes.total)
recipes.aromatic=(recipes.aromatic/recipes.total)
recipes.head()

C:\Users\ejfel\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,recipe,style,IBU,SRM,OG,FG,Mash,flaked,light_adjunct,dark_adjunct,...,black,brown,chocolate,crystal_150,roasted_barley,special,candi_180,carafa2,aromatic,total
0,byo_porter,0.0,35.0,50.0,58.0,14.0,153.0,0.0,0.035062,0.002922,...,0.002922,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,9.25
1,gf_porter,0.0,35.0,0.0,63.0,16.0,153.0,0.0,0.005487,0.019204,...,0.000000,0.016461,0.002743,0.000000,0.000000,0.0,0.0,0.0,0.0,13.50
2,bab_porter,0.0,40.0,0.0,0.0,0.0,153.0,0.0,0.014269,0.007134,...,0.002378,0.000000,0.004756,0.000000,0.000000,0.0,0.0,0.0,0.0,14.50
3,bs_porter,0.0,32.6,27.8,45.0,15.0,0.0,0.0,0.013061,0.009796,...,0.003265,0.000000,0.006531,0.000000,0.000000,0.0,0.0,0.0,0.0,8.75
4,la_porter,0.0,269.0,35.7,65.0,0.0,152.0,0.0,0.003327,0.011709,...,0.000000,0.000000,0.004191,0.006653,0.000865,0.0,0.0,0.0,0.0,12.26


In [80]:
profile = ProfileReport(recipes, title="Pandas Profiling Report on Stout/Porter Recipes")

C:\Users\ejfel\anaconda3\lib\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [81]:
profile.to_widgets()

In [82]:
features=recipes[['crystal_50','flaked','roasted_barley']]
target=recipes[['style']]

In [83]:
features.head()

,crystal_50,flaked,roasted_barley
0,0.000000,0.0,0.000000
1,0.005487,0.0,0.000000
2,0.004756,0.0,0.000000
3,0.009796,0.0,0.000000
4,0.000000,0.0,0.000865


In [84]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
X=features
y=target
clf = LogisticRegression(random_state=0).fit(X, y)

C:\Users\ejfel\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [90]:
clf.predict(X)

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 1.])

In [97]:
clf.predict(X_test)

array([0., 0.])

In [91]:
clf.score(X,y)

0.8125

In [92]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(random_state=0).fit(X,y)
regressor.predict(X)

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1.])

In [94]:
d = {'crystal_50': [0, .042], 'flaked': [0, 0], 'roasted_barley': [0, .042]}
X_test = pd.DataFrame(data=d)
X_test

,crystal_50,flaked,roasted_barley
0,0.000,0,0.000
1,0.042,0,0.042


In [96]:
regressor.predict(X_test)

array([0., 1.])